<a href="https://colab.research.google.com/github/Prakruthir12/ML_Project/blob/main/Credit_card_fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [4]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [5]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
13955,20631,1.504204,-0.411728,0.200090,-0.778753,-0.442232,-0.119677,-0.782660,-0.165178,0.691819,0.247322,-0.530014,-3.515087,2.173069,1.105206,0.231477,1.641043,0.401864,-0.601471,0.799229,0.079309,-0.136231,-0.217274,-0.143260,-1.057332,0.529188,-0.235062,-0.012089,0.000905,9.00,0.0
13956,20636,1.134994,0.096340,0.277921,0.319692,0.742800,1.611803,-0.458649,0.390012,1.424541,-0.656249,0.969943,-1.977200,2.131005,1.678891,1.480576,-0.682469,1.195737,-1.985939,-1.517055,-0.234249,-0.395605,-0.743542,0.222256,-1.859104,-0.109777,0.279049,0.012398,-0.009090,0.99,0.0
13957,20638,-6.305012,3.944886,-4.707362,1.539602,-3.934785,-1.730565,-2.104936,3.843447,0.863458,0.255574,-0.578456,-0.933662,1.902331,4.590772,-0.163232,0.562325,2.304264,0.308167,-0.304962,-0.077060,0.073140,-0.039935,-0.108896,0.691434,-0.261979,-0.447540,0.212900,-0.031021,89.99,0.0
13958,20638,1.161960,-0.398297,1.123732,-0.474237,-1.226667,-0.519325,-0.804179,0.070134,3.262926,-1.464389,0.814818,-2.218547,0.309302,1.577927,0.897422,-0.800820,1.053984,0.057109,-0.123652,-0.267050,-0.121191,0.097255,0.050903,0.330479,0.315692,-0.712765,0.073836,0.028055,11.85,0.0
13959,20642,1.291096,-0.226628,0.708386,-0.719236,-0.659099,-0.273757,-0.612042,-0.111488,3.032258,-1.478965,-0.094403,-1.584022,2.609250,1.066208,0.502725,-0.266146,0.233377,0.631159,0.660224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13960 entries, 0 to 13959
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    13960 non-null  int64  
 1   V1      13960 non-null  float64
 2   V2      13960 non-null  float64
 3   V3      13960 non-null  float64
 4   V4      13960 non-null  float64
 5   V5      13960 non-null  float64
 6   V6      13960 non-null  float64
 7   V7      13960 non-null  float64
 8   V8      13960 non-null  float64
 9   V9      13960 non-null  float64
 10  V10     13960 non-null  float64
 11  V11     13960 non-null  float64
 12  V12     13960 non-null  float64
 13  V13     13960 non-null  float64
 14  V14     13960 non-null  float64
 15  V15     13960 non-null  float64
 16  V16     13960 non-null  float64
 17  V17     13960 non-null  float64
 18  V18     13960 non-null  float64
 19  V19     13960 non-null  float64
 20  V20     13959 non-null  float64
 21  V21     13959 non-null  float64
 22

In [7]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       2
V27       2
V28       2
Amount    2
Class     2
dtype: int64

In [8]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    13905
1.0       53
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [9]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [10]:
print(legit.shape)
print(fraud.shape)

(13905, 31)
(53, 31)


In [11]:
# statistical measures of the data
legit.Amount.describe()

count    13905.000000
mean        62.617401
std        172.661337
min          0.000000
25%          5.910000
50%         15.950000
75%         51.790000
max       7712.430000
Name: Amount, dtype: float64

In [12]:
fraud.Amount.describe()

count      53.000000
mean      100.407925
std       318.684630
min         0.000000
25%         1.000000
50%         1.000000
75%         3.790000
max      1809.680000
Name: Amount, dtype: float64

In [13]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,7525.609996,-0.190759,0.256754,0.919049,0.259957,-0.056852,0.196485,-0.117536,-0.041800,0.948490,-0.249523,0.840659,-1.545288,1.021872,0.898954,-0.198743,0.020783,0.457342,-0.02264,-0.072848,0.024606,-0.075490,-0.177871,-0.026686,0.020377,0.102054,0.039295,0.002637,0.001893,62.617401
1.0,11435.509434,-3.741448,4.210880,-8.054355,6.319676,-2.313737,-1.954434,-5.621292,0.925182,-2.682759,-6.984619,6.590005,-10.285992,0.510247,-9.573349,-0.252661,-4.982615,-7.118854,-2.35123,0.220416,0.395363,0.293949,-0.038944,-0.330878,-0.400293,-0.139819,0.290936,0.680599,0.055737,100.407925


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [14]:
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [15]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [16]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1860,1438,-1.768929,1.208230,1.832726,-0.922455,0.213062,-0.803723,0.707416,-0.155229,0.358657,0.656303,1.446648,0.999284,0.111245,-0.457734,-0.888346,-0.013168,-0.606245,-0.324304,-0.076869,0.360090,-0.353946,-0.441542,-0.386752,0.572037,0.436299,0.336306,0.325751,0.029609,5.00,0.0
8784,8544,-6.571336,-11.943892,-4.246322,6.825816,-2.896624,2.706661,5.373923,-0.872187,-0.626203,-0.955356,0.951596,-2.136055,2.352625,2.669868,-1.412917,1.233990,0.038883,0.915151,-2.348414,7.922141,2.220493,-2.395098,-3.731391,-1.015761,-0.740824,-0.297883,-0.793710,0.680622,4002.88,0.0
8739,8458,1.259861,0.555062,0.473578,0.885370,-0.188364,-0.997164,0.077286,-0.353965,1.132149,-0.689272,1.090281,-1.621127,3.089313,0.903696,0.222252,0.391852,0.657196,-0.175947,-0.410024,-0.045239,-0.423910,-0.916018,0.126287,0.317854,0.234281,0.051683,-0.041843,0.027073,1.98,0.0
2198,1719,1.169806,0.188238,0.400703,1.394425,-0.179357,-0.190393,0.037884,0.007890,0.350622,-0.111892,-0.603415,0.320828,-0.523983,0.143231,-0.051253,-0.705097,0.285938,-0.930627,-0.364180,-0.220989,-0.107995,-0.087074,-0.054819,0.083982,0.650956,-0.311201,0.037833,0.014719,5.64,0.0
11042,12603,-2.591475,0.718685,0.553767,2.860996,1.596864,0.417922,0.696817,-0.255986,-0.429862,1.194257,1.183344,-2.333825,2.771619,1.677280,-0.222506,0.236437,0.078508,0.880063,1.793938,-1.158085,-0.484022,0.122769,0.541772,-0.949235,0.839878,0.370198,-0.489857,-0.544382,25.06,0.0


In [17]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
12898,18690,-15.398845,7.472324,-19.026912,11.165526,-6.893856,-2.120937,-14.913330,-0.721214,-7.175097,-14.166795,10.277769,-14.985434,0.345179,-14.666389,-0.346353,-8.333242,-12.602596,-4.876683,0.604626,1.111502,-2.444884,0.727495,-0.345078,-0.981749,0.995271,0.816762,2.262942,-1.178063,1.0,1.0
13344,19762,-14.179165,7.421370,-21.405836,11.927512,-7.974281,-2.202710,-15.471612,-0.356595,-6.380125,-13.348278,10.187587,-14.563980,0.593759,-14.491598,-0.281393,-8.264129,-12.439805,-4.837079,0.753150,1.002764,-2.366836,1.130955,0.991153,-1.033132,-0.327179,0.634693,2.171905,-1.395288,1.0,1.0
13711,20011,-14.724627,7.875157,-21.872317,11.906170,-8.348734,-2.262846,-15.833443,0.077874,-6.356833,-13.261652,10.063790,-14.394767,0.654889,-14.248316,-0.305361,-8.161632,-12.280965,-4.818586,0.719788,0.996469,-2.362345,1.099557,1.037199,-1.036359,-0.254777,0.642343,2.161129,-1.401282,1.0,1.0
13842,20332,-15.271362,8.326581,-22.338591,11.885313,-8.721334,-2.324307,-16.196419,0.512882,-6.333685,-13.175198,9.939820,-14.225456,0.716034,-14.004776,-0.329448,-8.059419,-12.122009,-4.799710,0.686227,0.993121,-2.356896,1.068019,1.085617,-1.039797,-0.182006,0.649921,2.149247,-1.406811,1.0,1.0
13881,20451,-15.819179,8.775997,-22.804686,11.864868,-9.092361,-2.386893,-16.560368,0.948349,-6.310658,-13.088891,9.815703,-14.056061,0.777192,-13.761018,-0.353636,-7.957447,-11.962954,-4.780508,0.652498,0.992279,-2.350634,1.036362,1.136051,-1.043414,-0.108923,0.657437,2.136424,-1.411945,1.0,1.0


In [18]:
new_dataset['Class'].value_counts()

0.0    492
1.0     53
Name: Class, dtype: int64

In [19]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,6893.713415,-0.165423,0.115658,0.958804,0.220723,-0.084915,0.214919,-0.101658,-0.042571,0.845227,-0.150749,0.776940,-1.549807,0.930251,0.819643,-0.257974,0.122687,0.386349,-0.005555,-0.021669,0.055842,-0.055080,-0.216093,-0.030537,0.040559,0.100847,0.008812,-0.025498,-0.016542,78.074858
1.0,11435.509434,-3.741448,4.210880,-8.054355,6.319676,-2.313737,-1.954434,-5.621292,0.925182,-2.682759,-6.984619,6.590005,-10.285992,0.510247,-9.573349,-0.252661,-4.982615,-7.118854,-2.351230,0.220416,0.395363,0.293949,-0.038944,-0.330878,-0.400293,-0.139819,0.290936,0.680599,0.055737,100.407925


Splitting the data into Features & Targets

In [20]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [21]:
print(X)

        Time         V1         V2  ...       V27       V28   Amount
1860    1438  -1.768929   1.208230  ...  0.325751  0.029609     5.00
8784    8544  -6.571336 -11.943892  ... -0.793710  0.680622  4002.88
8739    8458   1.259861   0.555062  ... -0.041843  0.027073     1.98
2198    1719   1.169806   0.188238  ...  0.037833  0.014719     5.64
11042  12603  -2.591475   0.718685  ... -0.489857 -0.544382    25.06
...      ...        ...        ...  ...       ...       ...      ...
12898  18690 -15.398845   7.472324  ...  2.262942 -1.178063     1.00
13344  19762 -14.179165   7.421370  ...  2.171905 -1.395288     1.00
13711  20011 -14.724627   7.875157  ...  2.161129 -1.401282     1.00
13842  20332 -15.271362   8.326581  ...  2.149247 -1.406811     1.00
13881  20451 -15.819179   8.775997  ...  2.136424 -1.411945     1.00

[545 rows x 30 columns]


In [22]:
print(Y)

1860     0.0
8784     0.0
8739     0.0
2198     0.0
11042    0.0
        ... 
12898    1.0
13344    1.0
13711    1.0
13842    1.0
13881    1.0
Name: Class, Length: 545, dtype: float64


Split the data into Training data & Testing Data

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [24]:
print(X.shape, X_train.shape, X_test.shape)

(545, 30) (436, 30) (109, 30)


Model Training

Logistic Regression

In [25]:
model = LogisticRegression()

In [26]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Model Evaluation

Accuracy Score

In [27]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [28]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9885321100917431


In [29]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [30]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.981651376146789
